In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np


In [2]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [3]:
data = pd.read_sql("select * from fire_and_air_data", con=engine)
data.head()

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,date,overall_aqi,site_name,year,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year
0,2001-01-01,153,San Francisco,2001,26892.0,LUCERNE,2001-01-01,2001-01-01,0.1,A,2001.0
1,2001-01-01,153,San Francisco,2001,27620.0,DEBRIS 2,2001-01-01,2001-01-01,0.1,A,2001.0
2,2001-01-01,153,San Francisco,2001,45869.0,LP HILL,2001-01-01,2001-01-01,0.1,A,2001.0
3,2001-01-01,153,San Francisco,2001,60502.0,PGE 1,2001-01-01,2001-01-01,0.1,A,2001.0
4,2001-01-01,153,San Francisco,2001,60589.0,PUMP HOUSE,2001-01-01,2001-01-01,0.1,A,2001.0


In [4]:
year = pd.read_sql("select distinct fire_year from fire_and_air_data", con=engine)
year.head()

,fire_year
0,2001.0
1,2000.0
2,NaN
3,2002.0
4,2003.0


In [ ]:
data = data[['date', 'overall_aqi', 'fire_id', 'size']]

In [ ]:
combined_group = data.groupby(['date','fire_id'])

In [ ]:
combined_group.head()

In [ ]:
# verify that there's never more then one record for a particular fire-id on a particular day
combined_group.size().max()

In [ ]:
grouped_by_date = data.groupby(['date'])

In [ ]:
grouped_by_date.describe().head()

In [ ]:
grouped_by_date.sum()

In [ ]:
acres_per_day_df = data.groupby('date')['size'].sum().to_frame().reset_index()

In [ ]:
acres_per_day_df.head()

In [ ]:
# Manually spot-check a date

data[data['date'] == '2014-01-03']

In [ ]:
air_data_df = pd.read_sql("select * from aqi_data", con=engine)
air_data_df.head()

In [ ]:
merged_data_df = pd.merge(air_data_df, acres_per_day_df, on='date')

In [ ]:
merged_data_df.head()

In [ ]:
merged_data_df.to_sql(name="acres_and_air_data", con=engine, if_exists="replace")